In [22]:
import numpy as np
import scipy.optimize
import matplotlib.pyplot as plt

import pandas as pd
import seaborn as sns

import matplotlib
matplotlib.rcParams.update({'font.size': 14})
from matplotlib import rc
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
rc('text', usetex=True)
rc('legend', fontsize=12)
matplotlib.rcParams['savefig.dpi'] = 1200

import pickle

%load_ext autoreload
%autoreload 2
import time

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
out_dir = "../../result/flu-adaptive/"
save_path = out_dir + "flu_adaptive-0728-10.pkl"
with open(save_path, 'rb') as handle:
    summary = pickle.load(handle)

In [24]:
def import_MS_FLU_pos_data():
    Y = np.loadtxt("../../data/flu.csv", delimiter=',')
    return Y

N = 50; 
num_mc = 10000
Y = import_MS_FLU_pos_data()
tau_ratio = -0.1
tau = np.mean(Y)*tau_ratio
prec_thres = N/tau**2 * 10e-3
var_thres = 1/prec_thres * 1e6

In [25]:
T_col = '${T}_\mathrm{max}$'; val_col = '$(\hat{\\tau}_0 - \\tau_0)^2$'; method_col = '$\mathrm{design}$'
method_dict = {'bm': '$\mathrm{benchmark}$', 'oracle': '$\mathrm{oracle}$', 'adaptive': '$\mathrm{adaptive}$', 'var': '$\mathrm{variance\,\,threshold\,\,} 1/c$'}
out = dict()
out[val_col] = list(); out[method_col] = list(); out[T_col] = list()


T_list = [7,14,21,28]
for T_max in T_list:
    this_out = summary[T_max] * 1e3
    num_mc = this_out.shape[0] 
    out[T_col] = out[T_col] + [T_max] * (num_mc * 4)
    out[method_col] = out[method_col] + [method_dict['bm']] * num_mc + [method_dict['oracle']] * num_mc + [method_dict['adaptive']] * num_mc + [method_dict['var']] * num_mc
    out[val_col] = out[val_col] + list(this_out['tau_bm'] ** 2) + list(this_out['tau_oracle'] ** 2) + list(this_out['tau_adaptive'] ** 2) + [var_thres] * num_mc
    
out_df = pd.DataFrame(out)

In [ ]:
fig, axes = plt.subplots(nrows=1,ncols=1,figsize=(7,4))
sns.lineplot(ax=axes, data=out_df, x=T_col, y=val_col, hue=method_col,estimator=np.mean, err_style="band")

axes.tick_params('x', labelrotation=360)
axes.set_xlabel(T_col)
axes.set_ylabel(val_col)
# axes.set_ylim([0.021, 0.057])
axes.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
axes.grid()

plt.tight_layout()

plt.savefig(out_dir+"supplementary/flu_adaptive_comparison-10.pdf")

In [ ]:
num_Ts = len(T_list)
fig, axes = plt.subplots(nrows=1,ncols=num_Ts,figsize=(4*num_Ts,4))

for i in range(num_Ts):
    T = T_list[i]
    if i == 0:
        axes[i].hist(summary[T]['T_ast'], bins= 15, color="whitesmoke", edgecolor="darkgrey", density=False)
#     elif i == 1 or i == 2:
#         axes[i].hist(summary[T]['T_ast'], bins= 25, color="whitesmoke", edgecolor="darkgrey", density=False)
    else:
        axes[i].hist(summary[T]['T_ast'], bins= T_list[-1], color="whitesmoke", edgecolor="darkgrey", density=False)
    axes[i].set_title("$T_{max}="+str(T)+"$")
    axes[i].set_xlim([1, 20])
    axes[i].set_ylim([0, 6501])
plt.tight_layout()
plt.savefig(out_dir+"supplementary/flu_termination_time-10.pdf")